In [284]:
from Transformer import EasyTransformer
from Transformer_config import EasyTransformerConfig
import torch
from dataclasses import dataclass
from typing import Optional, Callable
from torch.utils.data import Dataset, DataLoader
import pickle
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import torch.nn as nn
from train import *

/home/software/sloan/local/lib/py3.11/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
import psutil

# Get memory information
virtual_memory = psutil.virtual_memory()

# Convert from bytes to GB
total_memory_gb = virtual_memory.total / (1024 ** 3)
used_memory_gb = virtual_memory.used / (1024 ** 3)
available_memory_gb = virtual_memory.available / (1024 ** 3)

total_memory_gb, used_memory_gb, available_memory_gb

(1510.3419799804688, 358.103458404541, 1149.1404037475586)

In [247]:
cfg = EasyTransformerConfig(n_layers = 3,
            d_in = 5,
            d_model = 20,
            n_ctx = 43,
            d_head = 5,
            model_name = "custom",
            n_heads = 4,
            d_mlp = 64,
            act_fn = 'relu',
            eps = 1e-5,
            use_attn_scale = True, # whether to explicitly calculate the amount each head adds to the residual stream (with a hook) and THEN add it up, vs just calculating the sum. This can be very memory intensive for large models, so defaults to False
            init_mode = "gpt2",
            normalization_type = 'LN',
            device = "cuda" if torch.cuda.is_available() else "cpu",
            attention_dir = "causal",
            seed = 42,
            initializer_range = -1.0, # This will set: self.initializer_range = 0.8 / np.sqrt(self.d_model)
            positional_embedding_type = "standard",
            d_vocab_out = 3
        ) # For binary prediction

model = EasyTransformer(cfg)

NameError: name 'EasyTransformerConfig' is not defined

In [3]:
# Load the model's state dict onto CPU
model.load_state_dict(torch.load('../trained_models_DL/model_8000.pt', map_location=torch.device('cpu')))

/tmp/ipykernel_243676/1313021981.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('../trained_models_DL/model_8000.pt', map_location=torc

<All keys matched successfully>

In [ ]:
with open('../data/test_DL.pkl', 'rb') as file:
    test = pickle.load(file)

In [ ]:
test = test.iloc[-2500000:]

In [ ]:
num_features = 5
time_steps = 43

feature_cols = ['log_rets', 'high', 'low', '42_vol', 'volume']
lagged_feature_cols = [f"L_{lag}_{feat}" for lag in range(time_steps-1, 0, -1) for feat in feature_cols]
response_vars = [f'L_{l}_label' for l in range(42, 0, -1)] + ['label']

In [ ]:
test_dataset = StockDataset(test, lagged_feature_cols, feature_cols, response_vars)

dataloader = DataLoader(test_dataset, batch_size=2048, shuffle=True, collate_fn=collate_fn)

In [ ]:
criterion = nn.CrossEntropyLoss()
losses = 0

for i, batch in tqdm(enumerate(dataloader), total=len(test_dataset)//2048):
    inputs, targets = batch
    model.eval()
    outputs = model(inputs, return_type='logits')
    loss = criterion(outputs.view(-1, 3), targets.view(-1))
    losses += loss

In [13]:
i

106

In [14]:
losses/i

tensor(0.9195, grad_fn=<DivBackward0>)